In [1]:
import pandas as pd

In [2]:
print(pd.__file__)
print(pd.__version__)

/workspaces/data-engineering-zoomcamp-homework/01-docker-terraform/.venv/lib/python3.13/site-packages/pandas/__init__.py
3.0.0


In [3]:
folder = 'homework_data'
trip_df = pd.read_parquet(f'{folder}/green_tripdata_2025-11.parquet')
zone_df = pd.read_csv(f'{folder}/taxi_zone_lookup.csv')

In [4]:
trip_df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.2,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.0
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.2,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.0
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.5,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.0
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.7,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.0
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.4,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.0


In [5]:
trip_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 46912 entries, 0 to 46911
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               46912 non-null  int32         
 1   lpep_pickup_datetime   46912 non-null  datetime64[us]
 2   lpep_dropoff_datetime  46912 non-null  datetime64[us]
 3   store_and_fwd_flag     41343 non-null  str           
 4   RatecodeID             41343 non-null  float64       
 5   PULocationID           46912 non-null  int32         
 6   DOLocationID           46912 non-null  int32         
 7   passenger_count        41343 non-null  float64       
 8   trip_distance          46912 non-null  float64       
 9   fare_amount            46912 non-null  float64       
 10  extra                  46912 non-null  float64       
 11  mta_tax                46912 non-null  float64       
 12  tip_amount             46912 non-null  float64       
 13  tolls_amount

In [6]:
zone_df.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [7]:
zone_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   LocationID    265 non-null    int64
 1   Borough       265 non-null    str  
 2   Zone          264 non-null    str  
 3   service_zone  263 non-null    str  
dtypes: int64(1), str(3)
memory usage: 16.9 KB


In [22]:
november_2025_trip_df = trip_df[(trip_df['lpep_pickup_datetime'] >= '2025-11-01') & (trip_df['lpep_pickup_datetime'] < '2025-12-01')]

print(f"Number of trips has trip_distance less than or equal to 1 mile in November 2025: {(november_2025_trip_df['trip_distance'] <= 1).sum()}")

Number of trips has trip_distance less than or equal to 1 mile in November 2025: 8007


In [38]:
cleaned_trip_df = trip_df[trip_df['trip_distance'] < 100]
grouped_by_date_trip_df = cleaned_trip_df.groupby([cleaned_trip_df['lpep_pickup_datetime'].dt.date]).agg({'trip_distance': 'sum'}).reset_index().sort_values(by='trip_distance', ascending=False)
print(f"{grouped_by_date_trip_df.iloc[0]['lpep_pickup_datetime']} has the longest total trip distance of {grouped_by_date_trip_df.iloc[0]['trip_distance']} miles")

2025-11-20 has the longest total trip distance of 6377.03 miles


In [55]:
joined_df = pd.merge(trip_df, zone_df, how='left', left_on='PULocationID', right_on='LocationID').drop(columns=['LocationID']).rename(columns={'Zone': 'PUZone'})
joined_df = pd.merge(joined_df, zone_df, how='left', left_on='DOLocationID', right_on='LocationID').drop(columns=['LocationID']).rename(columns={'Zone': 'DOZone'})

In [56]:
november_18th_2025_joined_df = joined_df[joined_df['lpep_pickup_datetime'].dt.date == pd.to_datetime('2025-11-18').date()]
grouped_by_zone_joined_df = november_18th_2025_joined_df.groupby('PUZone').agg({'total_amount': 'sum'}).reset_index().sort_values(by='total_amount', ascending=False)
print(f"{grouped_by_zone_joined_df.iloc[0]['PUZone']} has the largest total amount on November 18th 2025 of {grouped_by_zone_joined_df.iloc[0]['total_amount']}")

East Harlem North has the largest total amount on November 18th 2025 of 9281.92


In [63]:
pu_east_harlem_north_november_2025_df = joined_df[
    (joined_df['PUZone'] == 'East Harlem North') &
    (joined_df['lpep_pickup_datetime'].dt.date >= pd.to_datetime('2025-11-01').date()) & 
    (joined_df['lpep_pickup_datetime'].dt.date < pd.to_datetime('2025-12-01').date())
]
pu_east_harlem_north_grouped_by_do_zone_df = pu_east_harlem_north_november_2025_df.groupby('DOZone').agg({'tip_amount': 'max'}).reset_index().sort_values(by='tip_amount', ascending=False)
print(f"{pu_east_harlem_north_grouped_by_do_zone_df.iloc[0]['DOZone']} has the largest total tip amount from East Harlem North of {pu_east_harlem_north_grouped_by_do_zone_df.iloc[0]['tip_amount']}")

Yorkville West has the largest total tip amount from East Harlem North of 81.89
